In [5]:
using JuMP, Cbc

┌ Info: Recompiling stale cache file C:\Users\lucasresck\.julia\compiled\v1.0\JuMP\DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1190
┌ Warning: Module Compat with build ID 87896584169362 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Warning: Module Compat with build ID 87896584169362 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Info: Precompiling MathProgBase [fdba3010-5040-5b88-9595-932c9decdf73]
└ @ Base loading.jl:1192
┌ Warning: Module Compat with build ID 87896584169362 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Info: Recompiling sta

In [12]:
Pkg.build("Xpress")

  Building Xpress → `C:\Users\lucasresck\.julia\packages\Xpress\Jdi4R\deps\build.log`


┌ Error: Error building `Xpress`: 
│ ERROR: LoadError: Unable to locate Xpress installation, please check your enviroment variable XPRESSDIR . Note that Xpress must be obtained separately from fico.com
│ Stacktrace:
│  [1] error(::String) at .\error.jl:33
│  [2] top-level scope at C:\Users\lucasresck\.julia\packages\Xpress\Jdi4R\deps\build.jl:40
│  [3] include at .\boot.jl:317 [inlined]
│  [4] include_relative(::Module, ::String) at .\loading.jl:1044
│  [5] include(::Module, ::String) at .\sysimg.jl:29
│  [6] include(::String) at .\client.jl:392
│  [7] top-level scope at none:0
│ in expression starting at C:\Users\lucasresck\.julia\packages\Xpress\Jdi4R\deps\build.jl:39
└ @ Pkg.Operations C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Pkg\src\Operations.jl:1097


In [10]:
N_t = 12
#José Batlle y Ordóñez (1), Punta del Tigre (1), La Tablada (2), Maldonado (1)
g_j = hcat([[394, 300, 285, 24] for i in 1:N_t]...)
#Salto Grande Dam (7), Constitución (El Palmar) Dam (3), Gabriel Terra (Rincón del Bonete) Dam (4), Baygorria Dam (3)
g_h = hcat([[1890, 333, 160, 108] for i in 1:N_t]...)
u_j = [1, 1, 2, 1]
u_h = [7, 3, 4, 3]
ψ_j = Int.(zeros(size(g_j)[1], N_t))
ψ_h = Int.(zeros(size(g_h)[1], N_t))
N_S = 10
r = zeros(2, N_S, N_t)
for s in 1:N_S
    for t in 1:N_t
        r[:, s, t] = [0, 0]
    end
end
d = Int.(round.(2494 .* [0.92, 0.93, 0.86, 0.87, 0.89, 1, 0.94, 0.96, 0.97, 0.89, 0.9, 0.91]))
d_s = zeros(N_S, N_t)
for s in 1:N_S
    for t in 1:N_t
        d_s[s, t] = d[t] - sum(r[:, s, t])
    end
end
β = 0.1
λ = 0.7
k = 1;

In [7]:
N_t = 3
#José Batlle y Ordóñez (1), Punta del Tigre (1), La Tablada (2), Maldonado (1)
g_j = hcat([[394, 300] for i in 1:N_t]...)
#Salto Grande Dam (7), Constitución (El Palmar) Dam (3), Gabriel Terra (Rincón del Bonete) Dam (4), Baygorria Dam (3)
g_h = hcat([[1890, 333, 160, 108] for i in 1:N_t]...)
u_j = [1, 1]
u_h = [7, 3]
ψ_j = Int.(zeros(size(g_j)[1], N_t))
ψ_h = Int.(zeros(size(g_h)[1], N_t))
N_S = 2
r = zeros(2, N_S, N_t)
for s in 1:N_S
    for t in 1:N_t
        r[:, s, t] = [0, 0]
    end
end
d = Int.(round.(2494 .* [0.92, 0.93, 0.86]))
d_s = zeros(N_S, N_t)
for s in 1:N_S
    for t in 1:N_t
        d_s[s, t] = d[t] - sum(r[:, s, t])
    end
end
β = 0.1
λ = 0.7
k = 1;

In [11]:
m = Model(solver = CbcSolver())

@variables m begin
    α_j[j = 1:size(g_j)[1], u = 1:u_j[j], t = 1:N_t], Bin
    α_h[h = 1:size(g_h)[1], u = 1:u_h[h], t = 1:N_t], Bin
    δ_s[1:N_S]
    z
    μ_s[1:N_S] <= 0
end

@constraintref reserve[1:N_t, 1:N_S]
for t in 1:N_t
    for s in 1:N_S
        reserve[t, s] = @constraint(m, δ_s[s] <= 1 / d[t] * 
            (sum(sum((1 - ψ_h[h, t]) * g_h[h, t] / u_h[h] * α_h[h, u, t] for u in 1:u_h[h]) for h in 1:size(g_h)[1])
                + sum(sum((1 - ψ_j[j, t]) * g_j[j, t] / u_j[j] * α_j[j, u, t] for u in 1:u_j[j]) for j in 1:size(g_j)[1])))
    end
end

@constraintref CVaR[1:N_S]
for s in 1:N_S
    CVaR[s] = @constraint(m, μ_s[s] + z <= δ_s[s])
end

@constraintref units_h[1:size(g_h)[1], 1:sort(u_h)[end]]
for h in 1:size(g_h)[1]
    for u in 1:u_h[h]
        units_h[h, u] = @constraint(m, sum(α_h[h, u, t] for t in 1:N_t) == N_t - k)
    end
end

@constraintref units_j[1:size(g_j)[1], 1:sort(u_j)[end]]
for j in 1:size(g_j)[1]
    for u in 1:u_j[j]
        units_j[j, u] = @constraint(m, sum(α_j[j, u, t] for t in 1:N_t) == N_t - k)
    end
end

@expression(m, CVaR_β, z + 1 / N_S * 1 / (1 - β) * sum(μ_s))

@objective(m, Max, λ * CVaR_β + (1 - λ) * 1 / N_S * sum(δ_s));

In [ ]:
@time solve(m)